In [1]:
from brt._C.router import generate_dst_indices, generate_src_indices
import torch
hot_mask = torch.randint(0, 2, (16, 4), dtype=torch.int32).cuda()
print(hot_mask)

tensor([[1, 0, 0, 1],
        [1, 0, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 1, 0],
        [1, 0, 1, 0],
        [1, 0, 1, 1],
        [1, 0, 0, 0],
        [1, 0, 1, 1],
        [0, 0, 1, 1],
        [0, 0, 1, 1],
        [0, 0, 1, 1],
        [0, 0, 1, 1],
        [0, 1, 0, 1],
        [1, 0, 1, 1],
        [0, 0, 1, 1],
        [1, 0, 1, 1]], device='cuda:0', dtype=torch.int32)


In [2]:
dst_indices, loads = generate_dst_indices(hot_mask)
src_indices, loads = generate_src_indices(hot_mask)
# print(indices[0].is_contiguous())
print(f"dst format: {dst_indices}")
print(f"src format: {src_indices}")



dst format: tensor([[ 1,  0,  0,  1],
        [ 2,  0,  0,  0],
        [ 3,  1,  0,  0],
        [ 4,  2,  1,  0],
        [ 5,  0,  2,  0],
        [ 6,  0,  3,  2],
        [ 7,  0,  0,  0],
        [ 8,  0,  4,  3],
        [ 0,  0,  5,  4],
        [ 0,  0,  6,  5],
        [ 0,  0,  7,  6],
        [ 0,  0,  8,  7],
        [ 0,  3,  0,  8],
        [ 9,  0,  9,  9],
        [ 0,  0, 10, 10],
        [10,  0, 11, 11]], device='cuda:0', dtype=torch.int32)
src format: tensor([[ 0,  2,  3,  0],
        [ 1,  3,  4,  5],
        [ 2, 12,  5,  7],
        [ 3,  0,  7,  8],
        [ 4,  0,  8,  9],
        [ 5,  0,  9, 10],
        [ 6,  0, 10, 11],
        [ 7,  0, 11, 12],
        [13,  0, 13, 13],
        [15,  0, 14, 14],
        [ 0,  0, 15, 15],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]], device='cuda:0', dtype=torch.int32)


In [3]:
from brt.router.utils import convert_index_format
shuffled_dst_indices = dst_indices[torch.randperm(dst_indices.size(0))]
print(shuffled_dst_indices)
loads = torch.sum(hot_mask, dim=0, dtype=torch.int32).cpu()
print(loads)
src_shuffled_dst_indices = convert_index_format(shuffled_dst_indices, loads, "dst_index", "src_index")

print(src_shuffled_dst_indices)

tensor([[ 6,  0,  3,  2],
        [ 8,  0,  4,  3],
        [10,  0, 11, 11],
        [ 0,  3,  0,  8],
        [ 0,  0,  6,  5],
        [ 0,  0,  8,  7],
        [ 1,  0,  0,  1],
        [ 9,  0,  9,  9],
        [ 2,  0,  0,  0],
        [ 0,  0,  5,  4],
        [ 4,  2,  1,  0],
        [ 7,  0,  0,  0],
        [ 5,  0,  2,  0],
        [ 3,  1,  0,  0],
        [ 0,  0, 10, 10],
        [ 0,  0,  7,  6]], device='cuda:0', dtype=torch.int32)
tensor([10,  3, 11, 11], dtype=torch.int32)
tensor([[ 6, 13, 10,  6],
        [ 8, 10, 12,  0],
        [13,  3,  0,  1],
        [10,  0,  1,  9],
        [12,  0,  9,  4],
        [ 0,  0,  4, 15],
        [11,  0, 15,  5],
        [ 1,  0,  5,  3],
        [ 7,  0,  7,  7],
        [ 2,  0, 14, 14],
        [ 0,  0,  2,  2],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]], device='cuda:0', dtype=torch.int32)


In [ ]:
placement_indices = torch.index_select(
    dst_indices, 1, torch.tensor([3, 2, 1, 0], dtype=torch.int32).cuda()
)
print(placement_indices)